# 03 - C-S-H Gel Formation Analysis

## 목표
- 10 ps MD 시뮬레이션 (C-S-H 젤 형성 관찰)
- Ca 용출 메커니즘 분석
- Si 배위 환경 변화 추적
- C-S-H 전구체 형성 지표 정량화
- H-bond 네트워크 분석

## 설정 옵션
- **빠른 테스트**: 2 ps (~20분)
- **전체 분석**: 10 ps (~3시간)

**현재 설정**: 아래 SIMULATION_TIME 변수로 조절

---
## Cell 1: Setup & Configuration
**실행 시간**: ~10초

In [ ]:
# ============================================================
# 설정 (여기만 수정하세요!)
# ============================================================
SIMULATION_TIME = 2.0  # ps - 빠른 테스트용 (20-30분)
# SIMULATION_TIME = 10.0  # ps - 최종 분석용 (3시간) - 주석 해제하고 위를 주석 처리

SAVE_INTERVAL = 10  # 몇 step마다 저장할지 (메모리 절약)
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import time
from datetime import datetime
import warnings

# ASE
from ase.io import read, write
from ase.io.trajectory import Trajectory

# CHGNet
from chgnet.model import CHGNet
from chgnet.model.dynamics import MolecularDynamics
import torch

warnings.filterwarnings('ignore')

# Matplotlib settings
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("="*70)
print("03 - C-S-H Gel Formation Analysis")
print("="*70)
print(f"Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\nSimulation Settings:")
print(f"  Duration: {SIMULATION_TIME} ps")
print(f"  Steps: {int(SIMULATION_TIME * 1000)}")
print(f"  Save interval: every {SAVE_INTERVAL} steps")
print(f"  Expected time: ~{int(SIMULATION_TIME * 10)} minutes (GPU)")
print(f"\nGPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

---
## Cell 2: Path Setup & Load Previous Results
**실행 시간**: ~2초

In [ ]:
# 경로 설정
WORK_DIR = Path("C:/cement_final")

FOLDERS = {
    'structures': WORK_DIR / "structures",
    'trajectories': WORK_DIR / "trajectories",
    'figures': WORK_DIR / "figures",
    'results': WORK_DIR / "results",
    'logs': WORK_DIR / "logs",
}

# Paper figures 폴더
paper_figs = FOLDERS['figures'] / 'paper'
paper_figs.mkdir(exist_ok=True)

print("\nFolders:")
for name, path in FOLDERS.items():
    print(f"  {name:15s}: {'✓' if path.exists() else '✗'}")

# 입력 파일 확인
input_structure = FOLDERS['structures'] / 'C3S_hydration_final.cif'
print(f"\nInput structure: {input_structure.exists()}")
if not input_structure.exists():
    print("  ✗ C3S_hydration_final.cif not found!")
    print("  → Run 01_Environment_Setup.ipynb first")
else:
    print(f"  ✓ Found ({input_structure.stat().st_size / 1024:.1f} KB)")

# 이전 결과 로드 (02번 노트북)
print("\nPrevious results (02번):")
prev_results = {
    'bond_analysis': FOLDERS['results'] / 'bond_analysis.json',
    'coordination': FOLDERS['results'] / 'coordination_analysis.json',
}

for name, path in prev_results.items():
    if path.exists():
        print(f"  ✓ {name}")
        if name == 'bond_analysis':
            with open(path, 'r') as f:
                bond_data = json.load(f)
                print(f"    Ca-O: {bond_data['Ca-O']['mean']:.3f} Å")
                print(f"    Si-O: {bond_data['Si-O']['mean']:.3f} Å")
    else:
        print(f"  ✗ {name} (optional)")

---
## Cell 3: Load CHGNet & Input Structure
**실행 시간**: ~5초

In [ ]:
# CHGNet 로드
print("\nLoading CHGNet...")
chgnet = CHGNet.load()
print(f"  CHGNet loaded")

# 구조 로드 (1 ps 수화 후)
print("\nLoading C3S hydration structure (after 1 ps)...")
atoms = read(input_structure)

print(f"  Atoms: {len(atoms)}")
print(f"  Formula: {atoms.get_chemical_formula()}")
print(f"  Volume: {atoms.get_volume():.2f} Å³")

# 원자 구성
symbols = np.array(atoms.get_chemical_symbols())
unique, counts = np.unique(symbols, return_counts=True)
print(f"\n  Composition:")
for sym, cnt in zip(unique, counts):
    print(f"    {sym}: {cnt}")

# 초기 에너지
prediction = chgnet.predict_structure(atoms)
initial_energy = prediction['e']
print(f"\n  Initial energy: {initial_energy:.4f} eV")
print(f"  Per atom: {initial_energy / len(atoms):.4f} eV/atom")

---
## Cell 4: Run MD Simulation
**실행 시간**: 20분 (2ps) ~ 3시간 (10ps)

⚠️ **주의**: 이 셀은 시간이 오래 걸립니다!

In [ ]:
# MD 설정
steps = int(SIMULATION_TIME * 1000)
timestep = 1.0  # fs
temperature = 300  # K

print("="*70)
print("MD Simulation Starting")
print("="*70)
print(f"  Steps: {steps}")
print(f"  Timestep: {timestep} fs")
print(f"  Temperature: {temperature} K")
print(f"  Total time: {SIMULATION_TIME} ps")
print(f"  Expected duration: ~{int(SIMULATION_TIME * 10)} minutes\n")

# 출력 파일
traj_path = FOLDERS['trajectories'] / f'csh_formation_{SIMULATION_TIME}ps.traj'
log_path = FOLDERS['logs'] / f'csh_formation_{SIMULATION_TIME}ps.log'

print(f"  Trajectory: {traj_path}")
print(f"  Log: {log_path}")
print("\n" + "="*70)

# MD 실행
md = MolecularDynamics(
    atoms=atoms,
    model=chgnet,
    ensemble="nvt",
    temperature=temperature,
    timestep=timestep,
    trajectory=str(traj_path),
    logfile=str(log_path),
    loginterval=SAVE_INTERVAL,
)

start_time = time.time()
print(f"\nStarted at: {datetime.now().strftime('%H:%M:%S')}")
print("\nProgress (every 5%):")

# Custom progress callback
progress_interval = max(1, steps // 20)  # 5% 단위

def print_progress(dyn):
    step = dyn.nsteps
    if step % progress_interval == 0 or step == steps:
        progress = step / steps * 100
        elapsed = time.time() - start_time
        if step > 0:
            eta = elapsed / step * (steps - step)
            eta_min = eta / 60
            print(f"  [{progress:5.1f}%] Step {step:5d}/{steps} | "
                  f"Elapsed: {elapsed/60:5.1f} min | ETA: {eta_min:5.1f} min")

# Attach observer
md.attach(print_progress, interval=progress_interval)

# Run!
md.run(steps=steps)

total_time = time.time() - start_time
print(f"\n{'='*70}")
print(f"✓ MD Simulation Complete!")
print(f"{'='*70}")
print(f"  Total time: {total_time/60:.1f} minutes ({total_time/3600:.2f} hours)")
print(f"  Speed: {total_time/steps*1000:.2f} ms/step")
print(f"  Trajectory saved: {traj_path}")
print(f"  Size: {traj_path.stat().st_size / 1024:.1f} KB")

---
## Cell 5: Load Trajectory & Basic Analysis
**실행 시간**: ~10초

In [ ]:
# 궤적 로드
print("\nLoading trajectory...")
traj = Trajectory(str(traj_path))
n_frames = len(traj)

print(f"  Frames: {n_frames}")
print(f"  Duration: {SIMULATION_TIME} ps")
print(f"  Time resolution: {SIMULATION_TIME / n_frames * 1000:.2f} fs/frame")

# 시간 배열
times = np.linspace(0, SIMULATION_TIME, n_frames)

# 첫/마지막 프레임 확인
first_frame = traj[0]
last_frame = traj[-1]

print(f"\nInitial (t=0):")
print(f"  Formula: {first_frame.get_chemical_formula()}")

print(f"\nFinal (t={SIMULATION_TIME}ps):")
print(f"  Formula: {last_frame.get_chemical_formula()}")

print("\n✓ Trajectory loaded successfully")

---
## Cell 6: Ca Leaching Analysis
**실행 시간**: ~30초

Ca 원자가 C3S lattice에서 벗어나는 과정 추적

In [ ]:
print("="*70)
print("Ca Leaching Analysis")
print("="*70)

# 초기 Ca 위치 (reference)
initial_atoms = traj[0]
initial_symbols = np.array(initial_atoms.get_chemical_symbols())
ca_mask = initial_symbols == 'Ca'
initial_ca_pos = initial_atoms.positions[ca_mask]
initial_ca_center = np.mean(initial_ca_pos, axis=0)

print(f"\nInitial Ca configuration:")
print(f"  Number of Ca: {np.sum(ca_mask)}")
print(f"  Center: [{initial_ca_center[0]:.2f}, {initial_ca_center[1]:.2f}, {initial_ca_center[2]:.2f}] Å")

# Ca leaching 판정 기준
LEACH_DISTANCE = 5.0  # Å (lattice 중심에서 이 거리 이상이면 용출)

print(f"  Leaching cutoff: {LEACH_DISTANCE} Å from center")

# 시간에 따른 Ca 용출 추적
ca_leached_count = []
ca_distances_from_center = []

print(f"\nAnalyzing {n_frames} frames...")
for i, atoms in enumerate(traj):
    if i % (n_frames // 10) == 0:
        print(f"  Progress: {i/n_frames*100:.0f}%")
    
    symbols = np.array(atoms.get_chemical_symbols())
    ca_pos = atoms.positions[symbols == 'Ca']
    
    # 각 Ca의 lattice 중심으로부터 거리
    distances = np.linalg.norm(ca_pos - initial_ca_center, axis=1)
    ca_distances_from_center.append(distances)
    
    # 용출된 Ca 개수
    leached = np.sum(distances > LEACH_DISTANCE)
    ca_leached_count.append(leached)

ca_leached_count = np.array(ca_leached_count)

print(f"\n✓ Analysis complete")
print(f"\nResults:")
print(f"  Initial leached Ca: {ca_leached_count[0]}")
print(f"  Final leached Ca: {ca_leached_count[-1]}")
print(f"  Maximum leached: {np.max(ca_leached_count)}")
print(f"  Leaching rate: {ca_leached_count[-1] / SIMULATION_TIME:.2f} Ca/ps")

---
## Cell 7: Si Coordination Analysis
**실행 시간**: ~20초

Si 배위 환경 변화 (SiO4 → SiO5/SiO3)

In [ ]:
print("="*70)
print("Si Coordination Analysis")
print("="*70)

SI_O_CUTOFF = 2.5  # Å

print(f"\nCutoff: Si-O < {SI_O_CUTOFF} Å")

# 시간에 따른 Si 배위수
si_coordination_numbers = []

print(f"\nAnalyzing {n_frames} frames...")
for i, atoms in enumerate(traj):
    if i % (n_frames // 10) == 0:
        print(f"  Progress: {i/n_frames*100:.0f}%")
    
    symbols = np.array(atoms.get_chemical_symbols())
    si_mask = symbols == 'Si'
    o_mask = symbols == 'O'
    
    si_indices = np.where(si_mask)[0]
    o_indices = np.where(o_mask)[0]
    
    frame_cns = []
    for si_idx in si_indices:
        cn = 0
        for o_idx in o_indices:
            dist = atoms.get_distance(si_idx, o_idx, mic=True)
            if dist < SI_O_CUTOFF:
                cn += 1
        frame_cns.append(cn)
    
    si_coordination_numbers.append(frame_cns)

si_coordination_numbers = np.array(si_coordination_numbers)

print(f"\n✓ Analysis complete")
print(f"\nResults:")
print(f"  Number of Si: {si_coordination_numbers.shape[1]}")
print(f"\n  Initial (t=0):")
unique_init, counts_init = np.unique(si_coordination_numbers[0], return_counts=True)
for cn, count in zip(unique_init, counts_init):
    print(f"    CN={cn}: {count} ({count/len(si_coordination_numbers[0])*100:.0f}%)")

print(f"\n  Final (t={SIMULATION_TIME}ps):")
unique_final, counts_final = np.unique(si_coordination_numbers[-1], return_counts=True)
for cn, count in zip(unique_final, counts_final):
    print(f"    CN={cn}: {count} ({count/len(si_coordination_numbers[-1])*100:.0f}%)")

# Average CN over time
avg_si_cn = np.mean(si_coordination_numbers, axis=1)
print(f"\n  Average CN change: {avg_si_cn[0]:.2f} → {avg_si_cn[-1]:.2f}")

---
## Cell 8: Ca-Si Distance Analysis (C-S-H Indicator)
**실행 시간**: ~40초

Ca-Si 거리 < 3.5 Å = C-S-H 전구체 형성 지표

In [ ]:
print("="*70)
print("Ca-Si Distance Analysis (C-S-H Indicator)")
print("="*70)

CSH_DISTANCE = 3.5  # Å - C-S-H characteristic distance

print(f"\nC-S-H cutoff: Ca-Si < {CSH_DISTANCE} Å")
print(f"  (Literature: C-S-H has Ca-Si = 3.0-3.5 Å)")

# 시간에 따른 Ca-Si 거리
ca_si_close_pairs = []  # < CSH_DISTANCE인 쌍 개수
ca_si_min_distances = []  # 최소 Ca-Si 거리
ca_si_mean_distances = []  # 평균 Ca-Si 거리

print(f"\nAnalyzing {n_frames} frames...")
for i, atoms in enumerate(traj):
    if i % (n_frames // 10) == 0:
        print(f"  Progress: {i/n_frames*100:.0f}%")
    
    symbols = np.array(atoms.get_chemical_symbols())
    ca_pos = atoms.positions[symbols == 'Ca']
    si_pos = atoms.positions[symbols == 'Si']
    
    # 모든 Ca-Si 쌍 거리
    distances = []
    for ca_p in ca_pos:
        for si_p in si_pos:
            # Minimum image
            delta = si_p - ca_p
            delta = atoms.cell.scaled_positions(delta.reshape(1, -1))[0]
            delta = delta - np.round(delta)
            delta = atoms.cell.cartesian_positions(delta.reshape(1, -1))[0]
            dist = np.linalg.norm(delta)
            distances.append(dist)
    
    distances = np.array(distances)
    
    # 통계
    close_pairs = np.sum(distances < CSH_DISTANCE)
    ca_si_close_pairs.append(close_pairs)
    ca_si_min_distances.append(np.min(distances))
    ca_si_mean_distances.append(np.mean(distances))

ca_si_close_pairs = np.array(ca_si_close_pairs)
ca_si_min_distances = np.array(ca_si_min_distances)
ca_si_mean_distances = np.array(ca_si_mean_distances)

print(f"\n✓ Analysis complete")
print(f"\nResults:")
print(f"  Initial Ca-Si pairs < {CSH_DISTANCE} Å: {ca_si_close_pairs[0]}")
print(f"  Final Ca-Si pairs < {CSH_DISTANCE} Å: {ca_si_close_pairs[-1]}")
print(f"  Maximum close pairs: {np.max(ca_si_close_pairs)}")
print(f"\n  Min Ca-Si distance:")
print(f"    Initial: {ca_si_min_distances[0]:.3f} Å")
print(f"    Final: {ca_si_min_distances[-1]:.3f} Å")
print(f"\n  Mean Ca-Si distance:")
print(f"    Initial: {ca_si_mean_distances[0]:.3f} Å")
print(f"    Final: {ca_si_mean_distances[-1]:.3f} Å")

# C-S-H formation indicator
if ca_si_close_pairs[-1] > ca_si_close_pairs[0]:
    print(f"\n  → C-S-H precursor formation detected! ✓")
elif ca_si_close_pairs[-1] == ca_si_close_pairs[0]:
    print(f"\n  → No significant C-S-H formation (extend simulation)")
else:
    print(f"\n  → Structure rearrangement observed")

---
## Cell 9: Integrated Visualization
**실행 시간**: ~15초

4-panel 통합 분석 그래프

In [ ]:
print("\nCreating integrated visualization...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# (a) Ca leaching
ax1 = axes[0, 0]
ax1.plot(times, ca_leached_count, 'o-', color='#E74C3C', 
         linewidth=2, markersize=4, alpha=0.8)
ax1.set_xlabel('Time (ps)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Number of Leached Ca', fontsize=11, fontweight='bold')
ax1.set_title('(a) Ca Leaching', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(bottom=0)

# (b) Si coordination
ax2 = axes[0, 1]
ax2.plot(times, avg_si_cn, 'o-', color='#3498DB', 
         linewidth=2, markersize=4, alpha=0.8)
ax2.axhline(4.0, color='gray', linestyle='--', linewidth=1, 
            alpha=0.5, label='Perfect SiO₄')
ax2.set_xlabel('Time (ps)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Average Si Coordination Number', fontsize=11, fontweight='bold')
ax2.set_title('(b) Si Coordination Evolution', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# (c) Ca-Si close pairs (C-S-H indicator)
ax3 = axes[1, 0]
ax3.plot(times, ca_si_close_pairs, 'o-', color='#2ECC71', 
         linewidth=2, markersize=4, alpha=0.8)
ax3.set_xlabel('Time (ps)', fontsize=11, fontweight='bold')
ax3.set_ylabel(f'Ca-Si Pairs < {CSH_DISTANCE} Å', fontsize=11, fontweight='bold')
ax3.set_title('(c) C-S-H Precursor Formation', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.set_ylim(bottom=0)

# (d) Ca-Si minimum distance
ax4 = axes[1, 1]
ax4.plot(times, ca_si_min_distances, 'o-', color='#F39C12', 
         linewidth=2, markersize=4, alpha=0.8)
ax4.axhline(CSH_DISTANCE, color='red', linestyle='--', linewidth=1.5, 
            alpha=0.7, label=f'C-S-H cutoff ({CSH_DISTANCE} Å)')
ax4.set_xlabel('Time (ps)', fontsize=11, fontweight='bold')
ax4.set_ylabel('Min Ca-Si Distance (Å)', fontsize=11, fontweight='bold')
ax4.set_title('(d) Closest Ca-Si Approach', fontsize=12, fontweight='bold')
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)

plt.tight_layout()

# 저장
overview_path = FOLDERS['figures'] / f'csh_formation_overview_{SIMULATION_TIME}ps.png'
plt.savefig(overview_path, dpi=300, bbox_inches='tight')
print(f"\n✓ Saved: {overview_path}")
plt.show()

# 논문용
paper_path = paper_figs / 'Fig3_CSH_formation.png'
plt.savefig(paper_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved (paper): {paper_path}")

---
## Cell 10: Save Results
**실행 시간**: ~5초

In [ ]:
print("\nSaving results...")

# 1. Ca leaching data
ca_leach_csv = FOLDERS['results'] / f'ca_leaching_{SIMULATION_TIME}ps.csv'
with open(ca_leach_csv, 'w') as f:
    f.write("time_ps,leached_count\n")
    for t, count in zip(times, ca_leached_count):
        f.write(f"{t:.6f},{count}\n")
print(f"  ✓ {ca_leach_csv}")

# 2. Si coordination
si_coord_csv = FOLDERS['results'] / f'si_coordination_{SIMULATION_TIME}ps.csv'
with open(si_coord_csv, 'w') as f:
    header = "time_ps," + ",".join([f"si_{i}_cn" for i in range(si_coordination_numbers.shape[1])]) + "\n"
    f.write(header)
    for t, cns in zip(times, si_coordination_numbers):
        line = f"{t:.6f}," + ",".join([str(cn) for cn in cns]) + "\n"
        f.write(line)
print(f"  ✓ {si_coord_csv}")

# 3. Ca-Si distances
ca_si_csv = FOLDERS['results'] / f'ca_si_distances_{SIMULATION_TIME}ps.csv'
with open(ca_si_csv, 'w') as f:
    f.write("time_ps,close_pairs,min_distance,mean_distance\n")
    for t, pairs, min_d, mean_d in zip(times, ca_si_close_pairs, 
                                         ca_si_min_distances, ca_si_mean_distances):
        f.write(f"{t:.6f},{pairs},{min_d:.6f},{mean_d:.6f}\n")
print(f"  ✓ {ca_si_csv}")

# 4. Summary metadata
summary = {
    'simulation': {
        'duration_ps': float(SIMULATION_TIME),
        'steps': int(steps),
        'frames': int(n_frames),
        'temperature_K': float(temperature),
    },
    'ca_leaching': {
        'initial': int(ca_leached_count[0]),
        'final': int(ca_leached_count[-1]),
        'maximum': int(np.max(ca_leached_count)),
        'rate_per_ps': float(ca_leached_count[-1] / SIMULATION_TIME),
    },
    'si_coordination': {
        'initial_avg': float(avg_si_cn[0]),
        'final_avg': float(avg_si_cn[-1]),
        'change': float(avg_si_cn[-1] - avg_si_cn[0]),
    },
    'csh_formation': {
        'cutoff_angstrom': float(CSH_DISTANCE),
        'initial_pairs': int(ca_si_close_pairs[0]),
        'final_pairs': int(ca_si_close_pairs[-1]),
        'max_pairs': int(np.max(ca_si_close_pairs)),
        'min_ca_si_distance_initial': float(ca_si_min_distances[0]),
        'min_ca_si_distance_final': float(ca_si_min_distances[-1]),
    },
}

summary_json = FOLDERS['results'] / f'csh_formation_summary_{SIMULATION_TIME}ps.json'
with open(summary_json, 'w') as f:
    json.dump(summary, f, indent=2)
print(f"  ✓ {summary_json}")

print("\n" + "="*70)
print("✓ ALL RESULTS SAVED!")
print("="*70)

---
## Cell 11: Summary & Interpretation
**실행 시간**: ~1초

In [ ]:
print("\n" + "="*70)
print("C-S-H GEL FORMATION ANALYSIS SUMMARY")
print("="*70)

print(f"\n[Simulation]")
print(f"  Duration: {SIMULATION_TIME} ps ({steps} steps)")
print(f"  Temperature: {temperature} K")
print(f"  Frames analyzed: {n_frames}")

print(f"\n[Ca Leaching]")
print(f"  Leached Ca (t=0): {ca_leached_count[0]}")
print(f"  Leached Ca (t={SIMULATION_TIME}ps): {ca_leached_count[-1]}")
print(f"  Net leaching: {ca_leached_count[-1] - ca_leached_count[0]} Ca atoms")
print(f"  Rate: {ca_leached_count[-1] / SIMULATION_TIME:.2f} Ca/ps")

if ca_leached_count[-1] > 0:
    print(f"  → Ca dissolution observed ✓")
else:
    print(f"  → No Ca leaching (extend simulation)")

print(f"\n[Si Coordination]")
print(f"  Initial avg CN: {avg_si_cn[0]:.2f}")
print(f"  Final avg CN: {avg_si_cn[-1]:.2f}")
print(f"  Change: {avg_si_cn[-1] - avg_si_cn[0]:+.2f}")

if abs(avg_si_cn[-1] - 4.0) < 0.1:
    print(f"  → SiO₄ tetrahedra maintained ✓")
else:
    print(f"  → Si environment change detected")

print(f"\n[C-S-H Formation]")
print(f"  Ca-Si pairs < {CSH_DISTANCE} Å:")
print(f"    Initial: {ca_si_close_pairs[0]}")
print(f"    Final: {ca_si_close_pairs[-1]}")
print(f"    Change: {ca_si_close_pairs[-1] - ca_si_close_pairs[0]:+d}")

if ca_si_close_pairs[-1] > ca_si_close_pairs[0]:
    print(f"  → C-S-H precursor formation! ✓✓✓")
    csh_percentage = (ca_si_close_pairs[-1] / (9 * 3)) * 100  # 9 Ca, 3 Si
    print(f"  → Estimated C-S-H formation: ~{csh_percentage:.1f}%")
elif ca_si_close_pairs[-1] == ca_si_close_pairs[0]:
    print(f"  → No net C-S-H formation")
    print(f"  → Recommendation: Run 10 ps simulation")

print(f"\n  Min Ca-Si distance:")
print(f"    Initial: {ca_si_min_distances[0]:.3f} Å")
print(f"    Final: {ca_si_min_distances[-1]:.3f} Å")
print(f"    Change: {ca_si_min_distances[-1] - ca_si_min_distances[0]:+.3f} Å")

print(f"\n[Generated Files]")
print(f"  Trajectory: {traj_path}")
print(f"  Log: {log_path}")
print(f"  Figures: {overview_path}")
print(f"  Results: {len(list(FOLDERS['results'].glob(f'*{SIMULATION_TIME}ps*')))} files")

print(f"\n[Next Steps]")
if SIMULATION_TIME < 10.0:
    print(f"  1. Change SIMULATION_TIME to 10.0 in Cell 1")
    print(f"  2. Re-run MD (Cell 4) for full analysis")
    print(f"  3. Compare 2 ps vs 10 ps results")
else:
    print(f"  1. Proceed to 04_Alternative_Binders.ipynb")
    print(f"  2. Screen Materials Project database")
    print(f"  3. Compare C-S-H with alternatives")

print(f"\n" + "="*70)
print(f"✓ ANALYSIS COMPLETE!")
print(f"="*70)
print(f"Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Summary

This notebook analyzed C-S-H gel formation through:
- Extended MD simulation (2-10 ps)
- Ca leaching from C3S lattice
- Si coordination environment changes
- C-S-H precursor formation (Ca-Si < 3.5 Å)

**Key Findings**:
- Ca dissolution rate
- SiO₄ stability
- C-S-H formation indicators

**Next**: 04_Alternative_Binders.ipynb - Materials Project screening